# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [1]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string
from pathlib import Path
import os

In [2]:
from mturk_processing_utils import (
    programmatically_review_HITs,
    get_dataset, 
    get_hit_id_dict, 
    parse_batch, 
    approved_parsed_batch_2_dicts,
    view_assignment,
    mturk,
    get_completetion_progress
)

In [3]:
PROJECT_ROOT_DIR = '/Users/nehasrikanth/Documents/paraphrase-nlu/'
dataset_name = ['atomic', 'snli', 'social']


raw_data_paths = {d: os.path.join(PROJECT_ROOT_DIR, f'raw-data/defeasible-nli/defeasible-{d}') for d in dataset_name}

### Social

In [4]:
social_batches = [
    'mturk_data/creation/social_dnli_annotation_examples_2.json',
]

dnli_social = get_dataset(raw_data_paths['social'], 'social')

Unique premise-hypothesis pairs: 7893 / 77016
Loaded 77016 nonempty train examples...(skipped 3314 examples)
Unique premise-hypothesis pairs: 979 / 9343
Loaded 9343 nonempty dev examples...(skipped 467 examples)
Unique premise-hypothesis pairs: 982 / 9439
Loaded 9439 nonempty test examples...(skipped 421 examples)


In [5]:
_, hit_id_2_example_id_social_2, _ = get_hit_id_dict(social_batches[0])

In [ ]:
get_completetion_progress(hit_id_2_example_id_social_1)

In [20]:
view_assignment('30LSNF239UV6CQPS8NUDWN5T502I27', dnli_social, hit_id_2_example_id_social_2)


Worker:  A3D943YJI95XZY AssignmentId:  30LSNF239UV6CQPS8NUDWN5T502I27
Premise:  
Hypothesis:  It's wrong to expect money out of people.
Update:  You let them borrow from you before
Update Type:  weakener

{'paraphrase_1': 'You gave them money previously. ',
 'paraphrase_2': 'They took stuff from you before.',
 'paraphrase_3': 'You lent them money in the past.'}


In [21]:
response = mturk.approve_assignment(
      AssignmentId='30LSNF239UV6CQPS8NUDWN5T502I27',
      RequesterFeedback='Thanks for doing a great job!',
)

In [ ]:
response = mturk.reject_assignment(
    AssignmentId='39LNWE0K4UWFW2YUR74J260LONXUI5',
    RequesterFeedback='These are not 3 paraphrases of the evidence sentence.'
)

In [ ]:
response = mturk.create_worker_block(
    WorkerId='A2B8YUXLFIDKUG',
    Reason='Worker is spamming.'
)